<a href="https://colab.research.google.com/github/osung/summary/blob/master/summarize_llm_lg_exa_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import os

from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize

In [171]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
import time

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [69]:
df = pd.read_pickle('/content/drive/MyDrive/colab_run/summary_output.pkl')
display(df.head())

,id,topic,text,ksic,max_scores,summary_new
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,..."


In [47]:
device = torch.device("cuda")
model_name = 'LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct'

In [48]:
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

config.json: 0.00B [00:00, ?B/s]

configuration_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [49]:
model = model.half()
model = model.to(device)

In [50]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [51]:
max_token_length = getattr(model.config, "max_position_embeddings", 512)

In [52]:
max_token_length

32768

In [53]:
get_new_tokens = 256
prompt_max = max(16, max_token_length - get_new_tokens)

In [54]:
prompt_max

32512

In [103]:
def get_summarize(text, ksic='어떤 산업') :
    role = f'너는 뉴스를 요약하는 요약 전문가야. 다음 문장은 {ksic}에 대한 뉴스 기사인데, 이것을 요약해줘.'
    print(role)

    chat = [
              #{"role": "tool_list", "content": ""},
              {"role": "system", "content": role},
              {"role": "user", "content": text},
    ]

    inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_dict=True, return_tensors="pt",
                                          truncation=True, max_length=prompt_max)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    output_ids = model.generate(**inputs, max_length=max_token_length, stop_strings=["<|endofturn|>", "<|stop|>"], tokenizer=tokenizer)

    responses = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

    _, _, answer = responses[0].partition("[|assistant|]\n")

    return answer.strip()

In [104]:
get_summarize(df.iloc[0]['text'])

너는 뉴스를 요약하는 요약 전문가야. 다음 문장은 어떤 산업에 대한 뉴스 기사인데, 이것을 요약해줘.


''

In [37]:
q = "I like  "
print("A:", chat_with_exaone(q))

A: LLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLL


In [59]:
def get_summarize(text) :
  role = "너는 뉴스를 요약하는 요약 전문가야. 다음 문장은 어떤 산업에 대한 뉴스 기사인데, 이것을 요약해줘. \
          존대말을 사용하지 말고, 문장을 ~다. 로 서술형으로 끝내도록 해줘."

  chat = [
      {
          "role" : "system",
          "content" : role
      },
      {
          "role" : "user",
          "content" : text
      }
  ]

  inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True,
                                         return_dict=True, return_tensor="pt",
                                         truncation=True, max_length=prompt_max)
  inputs = {k: v.to(device) for k, v in inputs.items()}

  output_ids = model.generate(**inputs, max_length=max_token_length, stop_strings=["<|endofturn|>", "<|stop|>"],
                              tokenizer=tokenizer)

  responses = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

  _, _, answer = responses[0].partition("assistant\n")

  return answer.strip()


In [60]:
article = '코로나 환자 늘어 일반 중환자 치료 공백 코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코로나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다. 의료 현장에서는 “코로나 확산으로 암 환자 등 일반 중환자 치료에 공백이 생기고 있다”는 목소리가 나온다. 18일 김범석 서울대 혈액종양내과 교수팀이 2019~2020년 서울대병원에서 사망한 암 환자 1456명을 분석한 결과 2019년 서울대병원 응급실에서 세상을 떠난 암 환자는 53명에서 2020년 99명으로 약 86% 대폭 증가한 것으로 나타났다. 말기 암 환자의 응급실 체류 시간도 늘었고, 환자의 생존을 위해 강제로 혈압을 높이는 승압제를 사용한 환자 수도 52.3%에서 59.2%로 약 7%포인트 늘었다. 심폐소생술을 받은 암 환자도 12.5%에서 16.3%로 증가했다. 김 교수는 “코로나 환자도 중요하지만, 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다”고 말했다.'

In [67]:
get_summarize(article)

AttributeError: 'list' object has no attribute 'to'

In [61]:
prompt = f"다음 기사를 간단히 요약해줘:\n\n{article}\n\n요약:"

In [62]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [63]:
outputs = model.generate(**inputs, max_new_tokens=200)

In [65]:
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [66]:
summary

'다음 기사를 간단히 요약해줘:\n\n코로나 환자 늘어 일반 중환자 치료 공백 코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코로나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다. 의료 현장에서는 “코로나 확산으로 암 환자 등 일반 중환자 치료에 공백이 생기고 있다”는 목소리가 나온다. 18일 김범석 서울대 혈액종양내과 교수팀이 2019~2020년 서울대병원에서 사망한 암 환자 1456명을 분석한 결과 2019년 서울대병원 응급실에서 세상을 떠난 암 환자는 53명에서 2020년 99명으로 약 86% 대폭 증가한 것으로 나타났다. 말기 암 환자의 응급실 체류 시간도 늘었고, 환자의 생존을 위해 강제로 혈압을 높이는 승압제를 사용한 환자 수도 52.3%에서 59.2%로 약 7%포인트 늘었다. 심폐소생술을 받은 암 환자도 12.5%에서 16.3%로 증가했다. 김 교수는 “코로나 환자도 중요하지만, 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다”고 말했다.\n\n요약:\n코로나 바이러스 확산으로 인해 위중증 환자가 증가하면서 일반 중환자 치료에 공백이 생기고 있다. 서울대병원의 연구에 따르면, 2020년 암 환자의 응급실 사망률이 2019년 대비 약 86% 증가했으며, 환자들의 생존을 위한 치료에도 차질이 빚어지고 있다. 이에 의료계에서는 코로나 환자 치료에 집중하면서 일반 중환자 치료가 소홀해지고 있다는 우려를 표명하고 있다.'

In [23]:
# 3. 텍스트 생성
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=200)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [86]:
prompt = df.iloc[0]['text']

In [114]:
def after_assistant(text: str) -> str:
    _, sep, tail = text.partition("[|assistant|]")
    return tail if sep else ""   # 구분자가 없으면 빈 문자열

In [142]:
def get_summarize(text) :

  role = "너는 뉴스를 요약하는 요약 전문가야. 다음 문장은 어떤 산업에 대한 뉴스 기사인데, 이것을 요약해줘. \
          존대말을 사용하지 말고, 문장을 ~다. 로 서술형으로 끝내도록 해줘."

  messages = [
    {"role": "system",
     "content": role},
    {"role": "user", "content": text}
  ]

  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
  )

  output = model.generate(
    inputs.to("cuda"),
    #eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=max_token_length,
    do_sample=False,
    stop_strings=["<|endofturn|>", "<|stop|>"],
    tokenizer=tokenizer
  )

  responses = tokenizer.decode(output[0], skip_special_tokens=True)

  return after_assistant(responses)

In [146]:
text = df.iloc[3]['text']

In [147]:
text

'태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람을 맞고 자란 태안산 양파가 본격적으로 출하되고 있다. 태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3942톤을 생산해 8억 원의 매출을 올리고 있다. 태안산 양파는 해안에 접한 천혜의 자연조건에서 자라 맛과 향이 진하고 속이 꽉 차 있어 맛과 품질면에서 최고급으로 인정받으며 전국의 소비자들에게 큰 인기를 끌고 있다. 올해는 겨울철 한파와 봄철 잦은 강우라는 악조건 속에도 철저한 재배 관리로 양호한 생육을 보여 생산량은 오히려 증가했으며 가격은 전년 수준과 비슷한 1kg당 450원-500원 선에 거래되고 있다. 군 관계자는 “군에서는 양파의 파종에서 수확까지 기계화 사업을 추진해 농업인구 감소와 노령화에 따른 만성적인 인력 부족 문제를 해결하고 있다”며 “앞으로도 기후변화에 대비한 재배방법 개선 등의 기술보급을 통해 농가 소득 향상에 도움이 될 수 있도록 온 힘을 기울이겠다”고 말했다.'

In [148]:
get_summarize(text)

'태안군에서 해풍을 맞고 자란 명품 양파 수확이 활발히 진행 중이다. 약 150개 농가가 76헥타르 면적에서 연간 3942톤의 양파를 생산해 8억 원의 매출을 올리고 있으며, 이 양파는 맛과 품질이 뛰어나 전국적으로 인기가 높다. 올해는 기후적 어려움에도 불구하고 재배 관리가 잘 돼 생산량이 증가했고, 가격은 전년도와 비슷한 수준을 유지하고 있다. 군은 농업의 기계화를 통해 인력 부족 문제를 해결하고 있으며, 향후 기후변화에 대응하는 재배 기술 보급을 통해 농가 소득 증대를 지원할 계획이다.'

In [149]:
df

,id,topic,text,ksic,max_scores,summary_new
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,..."
...,...,...,...,...,...,...
765021,NWRW1900000038.41782,T,네이버·다음 실시간 검색어 순위 ‘정부 요청 시 삭제’ 규정 운영했다 ㆍ자동완성·연...,NaN,0.000000,국내 양대 포털사이트인 네이버와 카카오(다음)가 정부 당국이 요청할 경우 실시간 급...
765022,NLRW2000000006.29284,T,"[미래사회포럼]최정준 동방문화대학원대 교수, ""소명 알고 완성하는게 천명 이루는 길...",NaN,0.000000,22일 열린 미래사회포럼 강연자로 나선 최정준 동방문화대학원대학교 미래예측학과 교수...
765023,NLRW2000000006.29247,T,"원더브라88원, 8월 8일 기념해 초대형 프로모션 '행사 가격은?' 라이프스타일 전...",NaN,0.000000,라이프스타일 전문기업 그리티가 푸쉬업 브라브랜드 '원더브라' 브랜드데이를 맞아 8일...
765024,NWRW1900000038.41262,T,마이클 코스털리츠 방한 “남다른 시각과 국가적 투자가 노벨상 만들어” ㆍ올해 노벨물...,NaN,0.000000,올해 노벨 물리학상 수상자 중 한 명인 마이클 코스털리츠 미국 브라운대 교수는 20...


In [162]:
df_max_scores = df[df['max_scores'] > 0.38]

In [163]:
df_max_scores

,id,topic,text,ksic,max_scores,summary_new
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,..."
...,...,...,...,...,...,...
382785,NWRW1900000001.25080,T,삼성엔… 고맙고도 두려운 '아이폰 대박' 삼성전자가 애플의 '아이폰' 때문에 웃고 ...,c264,0.380656,삼성전자가 애플의 '아이폰' 때문에 웃고 울고 있는데 애플의 아이폰은 지난 2분기 ...
382786,NWRW1900000010.5442,T,"삼성 갤럭시S9, 애플 아이폰에 완승… 유럽 6國 소비자 평가서 1위 차지 애플 아...",c264,0.380547,20일 업계에 따르면 삼성전자의 갤럭시S9 시리즈가 이탈리아·프랑스·포르투갈·스페인...
382787,NPRW1900000056.8650,T,갤럭시노트3로 스위스철도 발권 삼성전자 모바일 B2B(기업 간 거래) 사업 역량이 ...,c264,0.380265,삼성전자는 갤럭시노트3갤럭시S4를 비롯한 스마트폰과 갤럭시탭3 등 태블릿 디바이스가...
382788,NLRW2200000005.12347,T,"갤럭시S21 울트라5G, 모바일 월드 콩그레스2021서 ‘최고 스마트폰’ 선정 삼성...",c264,0.380160,삼성전자 갤럭시 S21 울트라 5G가 스페인 바르셀로나에서 개최된 모바일 월드 콩그...


In [165]:
len(df_max_scores['ksic'].value_counts())

41

In [172]:
df_max_scores['summary_exaone'] = df_max_scores['text'].progress_apply(get_summarize)

  0%|          | 0/984 [00:00<?, ?it/s]

/tmp/ipython-input-405031340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_max_scores['summary_exaone'] = df_max_scores['text'].progress_apply(get_summarize)


In [173]:
df_max_scores

,id,topic,text,ksic,max_scores,summary_new,summary_exaone
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...,스위스 제빵사협회가 창립 25주년을 기념해 세계에서 가장 긴 초콜릿 파이인 1221...
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...,고등어 어획량이 20년 동안 3분의 1로 급감했다. 1996년 41만5000톤에서 ...
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...,고령 우곡 지역의 수박 농가들이 올해 이상기후로 인해 수확량이 급감하고 수박 품질이...
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...,태안군에서 해풍을 맞고 자란 명품 양파 수확이 활발히 진행 중이다. 약 150개 농...
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,...",국내 친환경 농산물 생산량이 전체 농산물 생산량의 11.9%를 넘어서며 10%를 처...
...,...,...,...,...,...,...,...
382785,NWRW1900000001.25080,T,삼성엔… 고맙고도 두려운 '아이폰 대박' 삼성전자가 애플의 '아이폰' 때문에 웃고 ...,c264,0.380656,삼성전자가 애플의 '아이폰' 때문에 웃고 울고 있는데 애플의 아이폰은 지난 2분기 ...,삼성전자는 애플의 아이폰 덕분에 CPU와 낸드플래시 메모리 등 핵심 부품 판매로 큰...
382786,NWRW1900000010.5442,T,"삼성 갤럭시S9, 애플 아이폰에 완승… 유럽 6國 소비자 평가서 1위 차지 애플 아...",c264,0.380547,20일 업계에 따르면 삼성전자의 갤럭시S9 시리즈가 이탈리아·프랑스·포르투갈·스페인...,삼성전자의 갤럭시S9과 갤럭시S9+가 유럽 6개국의 소비자 평가에서 애플의 아이폰 ...
382787,NPRW1900000056.8650,T,갤럭시노트3로 스위스철도 발권 삼성전자 모바일 B2B(기업 간 거래) 사업 역량이 ...,c264,0.380265,삼성전자는 갤럭시노트3갤럭시S4를 비롯한 스마트폰과 갤럭시탭3 등 태블릿 디바이스가...,"삼성전자가 스위스 국영 철도청(SBB)에 갤럭시노트3, 갤럭시S4, 갤럭시탭3 등을..."
382788,NLRW2200000005.12347,T,"갤럭시S21 울트라5G, 모바일 월드 콩그레스2021서 ‘최고 스마트폰’ 선정 삼성...",c264,0.380160,삼성전자 갤럭시 S21 울트라 5G가 스페인 바르셀로나에서 개최된 모바일 월드 콩그...,삼성전자의 갤럭시 S21 울트라 5G가 모바일 월드 콩그레스 2021에서 최고 스마...


In [174]:
df_max_scores.to_pickle('df_max_scores_exaone.pkl')